In [14]:
import os
import sys
import json
import torch
import os
import json
import shutil
import math
import numpy as np
from tqdm import tqdm
from torch import nn
from pathlib import Path
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc
import pandas as pd

In [15]:
df = pd.read_csv('./wiki_100000.csv')
df = df[:100]

In [16]:
# df.column

In [17]:
# df['text'].values[0]

In [18]:
f = open('./train.txt','w',encoding='utf-8')
f1 = open('./test.txt','w',encoding='utf-8')
f2= open('./valid.txt','w',encoding='utf-8')
for index,value in enumerate(df['text'].values):
    value = value.strip()
    if value:
        if index < 50:
            f.write(value + '\n')
        elif index > 75:
            f1.write(value + '\n')
        else :  
            f2.write(value + '\n')

In [19]:
# Model
class n_gram(nn.Module):
    def __init__(self,n_class ,n_step,emb_size):
        super(n_gram, self).__init__()
#         pdb.set_trace()
        self.embed = nn.Embedding(n_class, emb_size)   # (vocab_size,n_dim)
        self.classify = nn.Sequential(
            nn.Linear(n_step * emb_size, 128),   
            nn.ReLU(True),
            nn.Linear(128, n_class)
        )
 
    def forward(self, x):
        voc_embed = self.embed(x)  # get word embedding context_size*n_dim
        voc_embed = voc_embed.view(1, -1)  # join two word vectors together  1*(context_size*n_dim)
        out = self.classify(voc_embed)   # 1*vocab_size
        return out
class NNLM(nn.Module):
    def __init__(self, n_class,n_step,emb_size,n_hidden):
        super(NNLM, self).__init__()
        self.n_step = n_step
        self.emb_size = emb_size
        self.n_hidden = n_hidden

        self.C = nn.Embedding(n_class, emb_size)
        self.w1 = nn.Linear(n_step * emb_size, n_hidden, bias=False)
        self.b1 = nn.Parameter(torch.ones(n_hidden))
        self.w2 = nn.Linear(n_hidden, n_class, bias=False)
        self.w3 = nn.Linear(n_step * emb_size, n_class, bias=False)
        
        

    def forward(self, X):
        X = self.C(X)
        X = X.view(-1, self.n_step * self.emb_size)
        Y1 = torch.tanh(self.b1 + self.w1(X))
        b2 = self.w3(X)
        Y2 = b2 + self.w2(Y1)
        return Y2

class TextRNN(nn.Module):
    def __init__(self, n_class,emb_size,n_hidden):
        super(TextRNN, self).__init__()
        self.C = nn.Embedding(n_class, embedding_dim=emb_size)
        self.rnn = nn.RNN(input_size=emb_size, hidden_size=n_hidden)
        self.W = nn.Linear(n_hidden, n_class, bias=False)
        self.b = nn.Parameter(torch.ones([n_class]))

    def forward(self, X):
        X = self.C(X)
        X = X.transpose(0, 1)
        outputs, hidden = self.rnn(X)
        outputs = outputs[-1] 
        model = self.W(outputs) + self.b
        return model

class TextRNN_attention(nn.Module):
    def __init__(self, n_class,emb_size,n_hidden):
        super(TextRNN_attention, self).__init__()
        
        self.C = nn.Embedding(n_class, embedding_dim=emb_size)
        self.rnn = nn.RNN(input_size=emb_size, hidden_size=n_hidden)
        self.W = nn.Linear(2 * n_hidden, n_class, bias=False)
        self.b = nn.Parameter(torch.ones([n_class]))

    def forward(self, X):
        X = self.C(X)
        X = X.transpose(0, 1)  # X: [n_step, batch_size, embeding size]
        outputs, hidden = self.rnn(X)
        output = outputs[-1]
        attention = []
        for it in outputs[:-1]:
            attention.append(torch.mul(it, output).sum(dim=1).tolist())
        attention = torch.tensor(attention)
        attention = attention.transpose(0, 1)
        attention = nn.functional.softmax(attention, dim=1).transpose(0, 1)
        # get soft attention
        attention_output = torch.zeros(outputs.size()[1], n_hidden)
        for i in range(outputs.size()[0] - 1):
            attention_output += torch.mul(attention[i], outputs[i].transpose(0, 1)).transpose(0, 1)
        output = torch.cat((attention_output, output), 1)
        model = torch.mm(output, self.W.weight.T) + self.b  #[batch_size, n_class]
        return model

In [20]:
def train_on_epoch(model, loss_function, optimizer, data_loader, device, epoch):
    model.train()
    accu_loss = torch.zeros(1).to(device) 
    optimizer.zero_grad()
    data_loader = tqdm(data_loader, file=sys.stdout)
    preds_all = []
    labels_all = []
    
    for step, data in enumerate(data_loader):
        input, target = data
        pred = model(input.to(device))
        loss = loss_function(pred, target.to(device))
        loss.backward()
        accu_loss += loss.detach()
        data_loader.desc = "[train epoch {}] loss: {:.3f} ppl: {:.3f} ".format(
            epoch,accu_loss.item() / (step + 1),math.exp(accu_loss.item() / (step + 1))
        )
        if not torch.isfinite(loss):
            print('WARNING: non-finite loss, ending training ', loss)
            sys.exit(1)
        optimizer.step()
        optimizer.zero_grad()
    return accu_loss.item() / (step + 1),math.exp(accu_loss.item() / (step + 1))

def evaluate(model, loss_function, data_loader, device, epoch):
    model.eval()

    accu_loss = torch.zeros(1).to(device)  # cumulative loss
    preds_all = []
    labels_all = []
    data_loader = tqdm(data_loader, file=sys.stdout)
    for step, data in enumerate(data_loader):
        input, target = data
        pred = model(input.to(device))
        loss = loss_function(pred, target.to(device))
        accu_loss += loss
        data_loader.desc = "[valid epoch {}] loss: {:.3f} ppl: {:.3f} ".format(
            epoch,
            accu_loss.item() / (step + 1),math.exp(accu_loss.item() / (step + 1))
        )
    return accu_loss.item() / (step + 1), math.exp(accu_loss.item() / (step + 1))

In [21]:
class MyDataSet(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __getitem__(self, item):
        input = self.inputs[item]
        target = self.targets[item]

        return input, target

    def __len__(self):
        return len(self.inputs)

In [22]:
train_path = './train.txt'
valid_path = './valid.txt'
test_path = './test.txt'


In [23]:
# A piece of text processed into words
train_words_path = './train_alpha.txt'#Processed as each line 'l i k e #' is written to the file
valid_words_path = './valid_alpha.txt'
test_words_path = './test_alpha.txt'
def process_words(origin_path, output_path):
    f = open(origin_path, 'r',encoding='utf-8')
    f_out = open(output_path,'w',encoding='utf-8')
    lines = f.readlines()
    word_list = []
    for sentence in lines:
        sentence = sentence.strip()
        if sentence == '':
            continue
        word_list += sentence.split()
    word_list = set(word_list)
    for word in word_list:
        if word.isalpha() and len(list(word)) > 4:
            f_out.write( ' '.join(list(word))+ ' #' + '\n')
process_words(train_path,train_words_path)
process_words(valid_path,valid_words_path)
process_words(test_path,test_words_path)


In [24]:
def generate_vocab(data_path):
        word_list = []
        f = open(data_path, 'r',encoding='utf-8')
        lines = f.readlines()
        for sentence in lines:
            word_list += sentence.split()
        word_list = list(set(word_list))
        word2index_dict = {w: i + 2 for i, w in enumerate(word_list)}
        word2index_dict['<PAD>'] = 0
        word2index_dict['<UNK>'] = 1
        word2index_dict = dict(sorted(word2index_dict.items(), key=lambda x: x[1]))  # sort
        index2word_dict = {index: word for word, index in word2index_dict.items()}
        json_str = json.dumps(word2index_dict, indent=4)
        with open(vocab_path, 'w') as json_file:
            json_file.write(json_str)

        return word2index_dict, index2word_dict

def generate_dataset(train_path, word2index_dict, n_step):
    def word2index(word):
        try:
            return word2index_dict[word]
        except:
            return 1
    input_list = []
    target_list = []
    f = open(train_path, 'r',encoding='utf-8')
    lines = f.readlines()
    for sentence in lines:
        sentence = sentence.strip()
        if sentence == '':
#             pdb.set_trace()
            continue
        word_list = sentence.split()
        if len(word_list) < n_step + 1: 
            word_list = ['<PAD>'] * (n_step + 1 - len(word_list)) + word_list
        index_list = [word2index(word) for word in word_list]
        for i in range(len(word_list) - n_step):
            input = index_list[i: i + n_step]
            target = index_list[i + n_step]

            input_list.append(torch.tensor(input))
            target_list.append(torch.tensor(target))
    dataset = MyDataSet(input_list, target_list)

    return dataset

In [25]:
class trainer():
    def __init__(self,n_step,epochs,train_path,valid_path,test_path,train_loader,test_loader,valid_loader,word2index_dict,index2word_dict,model_dic):
        self.epochs = epochs
        self.n_step = n_step
        self.n_hidden = 5  
        self.emb_size = 512 
        self.save_epoch = 1
        self.batch_size = 1
        self.lr = 1e-3
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path
        self.word2index_dict = word2index_dict
        self.index2word_dict = index2word_dict
        self.n_class = len(self.word2index_dict)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.valid_loader = valid_loader
        self.model_dic = model_dic
    def train_test_model(self,modelname): 
        models_path = './' + modelname
        if not os.path.exists(models_path):
            os.makedirs(models_path)
        model = self.model_dic[modelname].to(device)
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=self.lr)
        trains_loss = []
        trains_ppl = []
        valids_ppl = []
        valids_loss = []
        for epoch in range(self.epochs):
            train_loss,train_ppl= train_on_epoch(model=model,loss_function=loss_function,optimizer=optimizer,data_loader=self.train_loader,device=device,epoch=epoch)
            valid_loss,valid_ppl = evaluate(model=model,loss_function=loss_function,data_loader=self.valid_loader,device=device,epoch=epoch)
            trains_loss.append(train_loss)
            trains_ppl.append(train_ppl)
            valids_loss.append(valid_loss)
            valids_ppl.append(valid_ppl)
            if (epoch + 1) % self.save_epoch == 0:
                torch.save(model, os.path.join(models_path, f'weights-{epoch + 1}.ckpt'))
        test_loss,test_ppl = evaluate(model=model,loss_function=loss_function,data_loader=self.test_loader,device=device,epoch=epoch)
        return test_loss,test_ppl,trains_loss,valids_loss,trains_ppl,valids_ppl
    

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
def draw_loss(epochs,trains_loss,valids_loss,models_path):
    plt.plot( trains_loss, 'k--', label='train_loss', lw=2,color='g')
    plt.plot(valids_loss, 'k--', label='valid_loss', lw=2,color='r')
    plt.xlim([0,epochs])  
    plt.ylim([0, max(trains_loss)])
    plt.xlabel('epoch')
    plt.ylabel('loss')  
    plt.title('loss_curve')
    plt.savefig(models_path + '/loss.png')
    plt.cla()
    plt.clf()
#     plt.show()
    plt.close()
#     gc.collect()
def draw_ppl(epochs,trains_ppl,valids_ppl,models_path):
    plt.plot([i for i in range(epochs)], trains_ppl, 'k--', label='train_ppl', lw=2,color='g')
    plt.plot([i for i in range(epochs)], valids_ppl, 'k--', label='valid_ppl', lw=2,color='r')
    plt.xlim([0,epochs])  
    plt.ylim([0, max(trains_ppl)])
    plt.xlabel('epoch')
    plt.ylabel('ppl')  
    plt.title('ppl_curve')
    plt.savefig(models_path + '/ppl.png')
    plt.cla()
    plt.clf()
    plt.close()
#     gc.collect()

In [28]:
def generate_dataset_one(sentence, word2index_dict, n_step=5):
    def word2index(word):
        try:
            return word2index_dict[word]
        except:
            return 1  # <UNK>
    input_list = []
    target_list = []
    word_list = sentence.split()
    if len(word_list) < n_step + 1:  # Insufficient words in the sentence, padding
        word_list = ['<PAD>'] * (n_step + 1 - len(word_list)) + word_list
    index_list = [word2index(word) for word in word_list]
    input = index_list[-n_step: ]
    target = index_list[n_step]
    input_list.append(torch.tensor(input))
    target_list.append(torch.tensor(target))
    dataset = MyDataSet(input_list, target_list)
    return dataset

In [29]:
vocab_path = './vocab.json'
word2index_dict, index2word_dict = generate_vocab(train_words_path)
n_class = len(word2index_dict)  
# instantiated dataset
n_step = 2  # number of steps, n-1 in paper
emb_size = 512
n_hidden = 5
epochs = 3
batch_size = 1
save_epoch = 1  # Save the model every epoch
nw = min([os.cpu_count(), batch_size if  batch_size > 1 else 0, 8]) 
train_dataset = generate_dataset(train_words_path, word2index_dict, n_step)
valid_dataset = generate_dataset(valid_words_path, word2index_dict, n_step)
test_dataset = generate_dataset(test_words_path, word2index_dict, n_step)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True, pin_memory=True,num_workers=nw)
valid_loader = DataLoader(valid_dataset,batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=nw)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=nw)
model_dic = {"ngram_alpha":n_gram(n_class,n_step,emb_size), "TextRnn_alpha":TextRNN(n_class,emb_size,n_hidden),  "TextRnnAttention_alpha":TextRNN_attention(n_class,emb_size,n_hidden),"NNLM_alpha":NNLM(n_class,n_step,emb_size,n_hidden)}
mm1 = trainer(n_step,epochs,train_path,valid_path,test_path,train_loader,test_loader,valid_loader,word2index_dict,index2word_dict,model_dic)
test_loss_models_alpha = {}
test_ppl_models_alpha = {}
def main_alpha(model_name):
    models_path = './' + model_name
    test_loss, test_ppl,trains_loss,valids_loss,trains_ppl,valids_ppl = mm1.train_test_model(model_name)
    f_loss = open(models_path + '/trian_valid_loss.txt','w',encoding='utf-8')
    
    f_ppl = open(models_path + '/trian_valid_ppl.txt','w',encoding='utf-8')
    
    for loss_t,loss_v in zip(trains_loss,valids_loss):
        f_loss.write(str(loss_t) + '\t' + str(loss_v) + '\n')
        
    for ppl_t,ppl_v in zip(trains_loss,valids_loss):
        f_ppl.write(str(ppl_t) + '\t' + str(ppl_v) + '\n')
#     draw_loss(epochs,trains_loss,valids_loss,models_path)
#     draw_ppl(epochs1,trains_ppl,valids_ppl,models_path)
    test_loss_models_alpha[model_name] = test_loss
    test_ppl_models_alpha[model_name] = test_ppl
# word completion
def next_alpha_predict(test_model,sentence):
    model = torch.load(test_model).to(device)
    with open(vocab_path, "r") as f:
        word2index_dict = json.load(f)
    index2word_dict = {index: word for word, index in word2index_dict.items()}
    print('Enter letters: {}'.format(sentence))
    while sentence[-1] != '#':
        sentence_dataset = generate_dataset_one(sentence,word2index_dict=word2index_dict)
        data_loader = DataLoader(sentence_dataset,batch_size=1,shuffle=False,pin_memory=True,num_workers=nw)
        for step, data in enumerate(data_loader):
            input, target = data
            pred = model(input.to(device))
            preds = pred.data.max(1, keepdim=True)[1]
            sentence = sentence + ' ' + index2word_dict[preds.item()]
    print('Word completion： {}'.format(sentence))

# Word completion predicts the next candidate letter
def rank_alpha_predict(test_model,sentence):
    model = torch.load(test_model).to(device)
    with open(vocab_path, "r") as f:
            word2index_dict = json.load(f)
    index2word_dict = {index: word for word, index in word2index_dict.items()}
    sentence_dataset = generate_dataset_one(sentence,word2index_dict=word2index_dict)
    data_loader = DataLoader(sentence_dataset,batch_size=1,shuffle=False,pin_memory=True,num_workers=nw)
    for step, data in enumerate(data_loader):
        input, target = data
        pred = model(input.to(device))
        rank_ids = torch.argsort(pred,dim=1,descending=True)[:,:5].numpy()[0]
        rank_words = [index2word_dict[i] for i in rank_ids]
        print('Next letter candidate: {}'.format(rank_words))


In [30]:
main_alpha('NNLM_alpha')
main_alpha('ngram_alpha')
main_alpha('TextRnn_alpha')
main_alpha('TextRnnAttention_alpha')
test_loss_models_alpha
test_ppl_models_alpha

[valid epoch 2] loss: 2.559 ppl: 12.923 : 100%|████████████████████████████████| 28570/28570 [00:16<00:00, 1690.81it/s]


{'NNLM_alpha': 11.20437723120408,
 'ngram_alpha': 15.891757135551002,
 'TextRnn_alpha': 13.595520055159893,
 'TextRnnAttention_alpha': 12.92293164156136}

In [31]:
sentence = 'w h a t e'
test_model = './TextRnnAttention_alpha/weights-{}.ckpt'.format(epochs)
next_alpha_predict(test_model,sentence)
rank_alpha_predict(test_model,sentence)

Enter letters: w h a t e
Word completion： w h a t e d #
Next letter candidate: ['d', 'r', '#', 's', 'n']


In [45]:
sentence = 's t u d e n'
test_model = './TextRnnAttention_alpha/weights-{}.ckpt'.format(epochs)
next_alpha_predict(test_model,sentence)
rank_alpha_predict(test_model,sentence)

Enter letters: s t u d e n
Word completion： s t u d e n #
Next letter candidate: ['#', 'a', 'e', 's', 't']


In [40]:
sentence = 'w a c t h'
test_model = './TextRnnAttention_alpha/weights-{}.ckpt'.format(epochs)
next_alpha_predict(test_model,sentence)
rank_alpha_predict(test_model,sentence)

Enter letters: w a c t h
Word completion： w a c t h e d #
Next letter candidate: ['e', 'i', 'a', 'n', 'o']


In [46]:
vocab_path = './vocab.json'
word2index_dict, index2word_dict = generate_vocab(train_path)
n_class = len(word2index_dict)  
# instantiated dataset
n_step = 5  
emb_size = 128
n_hidden = 5
epochs = 1
batch_size = 1
nw = min([os.cpu_count(), batch_size if  batch_size > 1 else 0, 8]) 
train_dataset = generate_dataset(train_path, word2index_dict, n_step)
valid_dataset = generate_dataset(valid_path, word2index_dict, n_step)
test_dataset = generate_dataset(test_path, word2index_dict, n_step)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True, pin_memory=True,num_workers=nw)
valid_loader = DataLoader(valid_dataset,batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=nw)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True,pin_memory=True,num_workers=nw)
model_dic = {"ngram":n_gram(n_class,n_step,emb_size), "TextRnn":TextRNN(n_class,emb_size,n_hidden),  "TextRnnAttention":TextRNN_attention(n_class,emb_size,n_hidden),"NNLM":NNLM(n_class,n_step,emb_size,n_hidden)}
mm1 = trainer(n_step,epochs,train_path,valid_path,test_path,train_loader,test_loader,valid_loader,word2index_dict,index2word_dict,model_dic)
test_loss_models = {}
test_ppl_models = {}
def main_words(model_name):
    models_path = './' + model_name
    test_loss, test_ppl,trains_loss,valids_loss,trains_ppl,valids_ppl = mm1.train_test_model(model_name)
    f_loss = open(models_path + '/trian_valid_loss.txt','w',encoding='utf-8')
    
    f_ppl = open(models_path + '/trian_valid_ppl.txt','w',encoding='utf-8')
    
    for loss_t,loss_v in zip(trains_loss,valids_loss):
        f_loss.write(str(loss_t) + '\t' + str(loss_v) + '\n')
        
    for ppl_t,ppl_v in zip(trains_loss,valids_loss):
        f_ppl.write(str(ppl_t) + '\t' + str(ppl_v) + '\n')
        
    test_loss_models[model_name] = test_loss
    test_ppl_models[model_name] = test_ppl
# next word prediction
def test_next_word(test_model,sentence):
    print('Enter a sentence: {}'.format(sentence))
    model = torch.load(test_model).to(device)
    with open(vocab_path, "r") as f:
            word2index_dict = json.load(f)
    index2word_dict = {index: word for word, index in word2index_dict.items()}
    count = 0
    while count < 10:
        count += 1
        sentence_dataset = generate_dataset_one(sentence,word2index_dict=word2index_dict)
        data_loader = DataLoader(sentence_dataset,batch_size=1,shuffle=False,pin_memory=True,num_workers=nw)
        for step, data in enumerate(data_loader):
            input, target = data
            pred = model(input.to(device))
            preds = pred.data.max(1, keepdim=True)[1]
            sentence = sentence + ' ' + index2word_dict[preds.item()]
        if count == 10:
            print('Predict the word ten times ahead：{}'.format(sentence))
def rank_words_predict(test_model,sentence):
    model = torch.load(test_model).to(device)
    with open(vocab_path, "r") as f:
            word2index_dict = json.load(f)
    index2word_dict = {index: word for word, index in word2index_dict.items()}
    count = 0
    sentence_dataset = generate_dataset_one(sentence,word2index_dict=word2index_dict)
    data_loader = DataLoader(sentence_dataset,batch_size=1,shuffle=False,pin_memory=True,num_workers=nw)
    for step, data in enumerate(data_loader):
        input, target = data
        pred = model(input.to(device))
        preds = pred.data.max(1, keepdim=True)[1]
        rank_ids = torch.argsort(pred,dim=1,descending=True)[:,:5].numpy()[0]
        rank_words = [index2word_dict[i] for i in rank_ids]
        print('Next word candidate: {}'.format(rank_words))
        sentence = sentence + ' ' + index2word_dict[preds.item()]


In [47]:
main_words('NNLM')
main_words('ngram')
main_words('TextRnnAttention')
main_words('TextRnn')

[valid epoch 0] loss: 11.639 ppl: 113426.148 : 100%|████████████████████████████| 25560/25560 [00:29<00:00, 870.27it/s]


In [53]:
sentence = 'as the dow jones industrial it' 
test_model = './ngram/weights-{}.ckpt'.format(epochs)
test_next_word(test_model,sentence)
rank_words_predict(test_model,sentence)

Enter a sentence: as the dow jones industrial it
Predict the word ten times ahead：as the dow jones industrial it the the the the the the the the the the
Next word candidate: ['the', 'of', 'and', 'in', 'to']


In [51]:
# was commissioned by
# a single Macchi crash landed near Zemun airfield

sentence = 'a single Macchi crash landed near' 
test_model = './TextRnn/weights-{}.ckpt'.format(epochs)
test_next_word(test_model,sentence)
rank_words_predict(test_model,sentence)

Enter a sentence: a single Macchi crash landed near
Predict the word ten times ahead：a single Macchi crash landed near the the the the the the the the the the
Next word candidate: ['the', 'in', 'and', 'of', 'to']


In [55]:
#  was contacted by Maxim and was given a photo shoot that appeared in the men's magazine.
sentence = 'was contacted by Maxim and was given a photo shoot that appeared' 
test_model = './ngram/weights-{}.ckpt'.format(epochs)
test_next_word(test_model,sentence)
rank_words_predict(test_model,sentence)

Enter a sentence: was contacted by Maxim and was given a photo shoot that appeared
Predict the word ten times ahead：was contacted by Maxim and was given a photo shoot that appeared the the the the the the the the the the
Next word candidate: ['the', 'a', 'in', 'of', 'to']


In [60]:
sentence = 'Nice to meet you' 
test_model = './NNLM/weights-{}.ckpt'.format(epochs)
test_next_word(test_model,sentence)
rank_words_predict(test_model,sentence)

Enter a sentence: Nice to meet you
Predict the word ten times ahead：Nice to meet you to He was the by saying, on the John and
Next word candidate: ['to', 'with', 'after', 'which', 'the']


In [62]:
sentence = 'Happy Birthday to' 
test_model = './NNLM/weights-{}.ckpt'.format(epochs)
test_next_word(test_model,sentence)
rank_words_predict(test_model,sentence)

Enter a sentence: Happy Birthday to
Predict the word ten times ahead：Happy Birthday to be able to the enough of the latter I and
Next word candidate: ['be', 'Creek', 'work', 'Notes', 'she']


In [64]:
sentence = 'I am glad that you' 
test_model = './NNLM/weights-{}.ckpt'.format(epochs)
test_next_word(test_model,sentence)
rank_words_predict(test_model,sentence)

Enter a sentence: I am glad that you
Predict the word ten times ahead：I am glad that you the Her is and with the of the Stafford of
Next word candidate: ['the', 'his', 'that', 'Stafford', 'not']
